In [1]:
import sys
sys.path.append('../')

In [2]:
import torch
import torch.nn as nn

from models.refinenet_resnet import refinenet_resnet101

pretrained_weight_dir = '../../refinenet-pytorch/pretrained/Cityscapes'
model = refinenet_resnet101(pretrained_weight_dir).eval()
model_name = '{}({})'.format(str(model).split('(')[0], str(model.resnet).split('(')[0])
print(model_name)

criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-5)

model = nn.DistributedDataParallel(model.cuda(), device_ids=[0, 1]).train()

RefineNet_ResNet(Resnet101)


In [3]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import transforms

import datasets

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda img: img*255)])
target_transform = transforms.Lambda(lambda img: torch.from_numpy(np.array(img)))

foggy_cityscapes = datasets.EnvCityscapes(
    '/workbench/data/Cityscapes', split='train',
    env=["", 'foggyDBF', 'transmittanceDBF'],
    env_type=[[''], ['beta_0.01'], ['beta_0.01']],
    target_type=['semantic', 'color'],
    transform=transforms.Lambda(lambda imgs: tuple(map(transform, imgs))),
    target_transform=transforms.Lambda(lambda imgs: tuple(map(target_transform, imgs))),
)
foggy_cityscapes_val = datasets.EnvCityscapes(
    '/workbench/data/Cityscapes', split='val',
    env=["", 'foggyDBF', 'transmittanceDBF'],
    env_type=[[''], ['beta_0.01'], ['beta_0.01']],
    target_type=['semantic', 'color'],
    transform=transforms.Lambda(lambda imgs: tuple(map(transform, imgs))),
    target_transform=transforms.Lambda(lambda imgs: tuple(map(target_transform, imgs))),
)
zurich_testv2 = datasets.Zurich(
    '/workbench/data/Foggy_Zurich', split='testv2',
    target_type=['gt_labelIds', 'RGB'],
    transform=transform,
    target_transform=transforms.Lambda(lambda imgs: tuple(map(target_transform, imgs))),
)

with open('/workbench/data/Cityscapes/foggy_trainval_refined_filenames.txt') as f:
    trainval_refined_filenames = sorted([line.strip() for line in f.readlines()])

images, targets = map(np.array, [foggy_cityscapes.images, foggy_cityscapes.targets])
refine_mask = np.zeros(images.shape[0]).astype(np.bool)
for i, image in enumerate(images):
    for filename in trainval_refined_filenames:
        if filename in image[0]:
            refine_mask[i] = True
            break
foggy_cityscapes.images, foggy_cityscapes.targets = images[refine_mask], targets[refine_mask]

images, targets = map(np.array, [foggy_cityscapes_val.images, foggy_cityscapes_val.targets])
refine_mask = np.zeros(images.shape[0]).astype(np.bool)
for i, image in enumerate(images):
    for filename in trainval_refined_filenames:
        if filename in image[0]:
            refine_mask[i] = True
foggy_cityscapes_val.images, foggy_cityscapes_val.targets = images[refine_mask], targets[refine_mask]

dataset_name = str(foggy_cityscapes).split()[1]
print(dataset_name)

train_loader = DataLoader(foggy_cityscapes)
valid_loader = DataLoader(foggy_cityscapes_val)
test_loader = DataLoader(zurich_testv2)

EnvCityscapes


In [4]:
import numpy as np
import torch.nn.functional as F

import datasets

classes = datasets.Cityscapes.classes
id2label = {cls.id:cls for cls in classes}
train_id2label = {cls.train_id:cls for cls in classes}
pred_argmax2label = {cls.train_id:cls for cls in classes}
pred_argmax2label[19] = pred_argmax2label[255]
pred_argmax2label.pop(255);

def foggy2data(foggy):
    data = F.interpolate(foggy, None, 0.7, mode='bilinear')-128
    return data
def labelId2gt(labelId):
    gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
    gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
    gt = torch.from_numpy(gt)
    return gt


In [6]:
import torch.nn.functional as F

import wandb

wandb.init(project='refinenet-pytorch', name=model_name, tags=dataset_name)

def wandb_scores(scores, split):
    for k, v in scores[0].items():
        wandb.log({'Valid_{}'.format(k): v}, commit=False)
    wandb.run.summary['{}_IoU'.format(split).format(split, k)] = list(scores[1].values())

def train_model(model, optimizer, criteria, data, gt):
    model.train()
    
    optimizer.zero_grad()
    logit = model(data).to(gt.device)
    scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
    loss = criteria(scaled_logit, gt)
    loss.backward()
    optimizer.step()
    return loss

def eval_model(model, running_metrics, data, gt):
    model.eval()
    with torch.no_grad():
        logit = model(data).to(gt.device)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
        loss = criteria(scaled_logit, gt)
        
        running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
    return loss

def eval_model_valid_loader(model, valid_loader, step, commit):
    epoch_loss = 0
    running_metrics = runningScore(20)
    for i, ((clear, foggy, trans), (labelId, color)) in enumerate(tqdm(valid_loader)):
        loss = eval_model(model, running_metrics,
                          foggy2data(foggy),
                          labelId2gt(labelId))
        epoch_loss += loss
        
    scores = running_metrics.get_scores()
    
    wandb_scores(scores, 'Valid')
    wandb.log({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)}, step=step, commit=commit)

    return epoch_loss / len(valid_loader), scores

def eval_model_test_loader(model, test_loader, step, commit):
    epoch_loss = 0
    running_metrics = runningScore(20)
    for i, (foggy, (labelId, color)) in enumerate(tqdm(test_loader)):
        loss = eval_model(model, running_metrics,
                          foggy2data(foggy),
                          labelId2gt(labelId))
        epoch_loss += loss
        
    scores = running_metrics.get_scores()
    
    wandb_scores(scores, 'Test')
    wandb.log({'Test_Epoch_Loss': epoch_loss / len(test_loader)}, step=step, commit=commit)
    
    return epoch_loss / len(test_loader), scores
    

W&B Run: https://app.wandb.ai/yonyeoseok/refinenet-pytorch/runs/108n8mgj

In [ ]:
import os
from tqdm import tqdm

from utils.metrics import runningScore

epoch_loss, scores = eval_model_valid_loader(model, valid_loader, step=0, commit=False)
epoch_loss, scores = eval_model_test_loader(model, test_loader, step=0, commit=True)

for epoch in range(1, 9):
    epoch_loss = 0
    for i, ((clear, foggy, trans), (labelId, color)) in enumerate(tqdm(train_loader)):
        loss = train_model(model, optimizer, criteria,
                           foggy2data(foggy),
                           labelId2gt(labelId))
        epoch_loss += loss
        wandb.log({'Train_Batch_Loss': loss}, step=(i+1) + len(train_loader)*(epoch-1))
    torch.save(model.state_dict(), os.path.join(wandb.run.dir, 'state_dict.{:02d}.pth').format(epoch))

    epoch_loss, scores = eval_model_valid_loader(model, valid_loader, step=len(train_loader)*epoch, commit=False)
    epoch_loss, scores = eval_model_test_loader(model, test_loader, step=len(train_loader)*epoch, commit=True)

  0%|          | 0/52 [00:00<?, ?it/s]/opt/conda/envs/py37torch1.5/lib/python3.8/site-packages/torch/nn/functional.py:2970: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "
/opt/conda/envs/py37torch1.5/lib/python3.8/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/opt/conda/conda-bld/pytorch_158742820743

    
        
    with torch.no_grad():
        epoch_loss = 0
        for i, ((clear, foggy, trans), (labelId, color)) in enumerate(tqdm(valid_loader)):
            logit = model(F.interpolate(foggy, None, 0.7, mode='bilinear')-128).cpu()
            gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
            gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
            gt = torch.from_numpy(gt)
            scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
            loss = criteria(scaled_logit, gt)
            
            epoch_loss += loss
                        
            foggy_cityscapes_val_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#             break
        wandb.log({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})
        
        scores = foggy_cityscapes_val_running_metrics.get_scores()
        for k, v in scores[0].items():
            wandb.log({'Valid_{}'.format(k): v})
        wandb.log({'Valid_IoU': list(scores[1].values())})
        
        epoch_loss = 0
        for i, (foggy, (labelId, color)) in enumerate(tqdm(test_loader)):
            logit = model(F.interpolate(foggy, None, 0.7, mode='bilinear')-128).cpu()
            gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
            gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
            gt = torch.from_numpy(gt)
            scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
            loss = criteria(scaled_logit, gt)
            
            epoch_loss += loss
            
            foggy_zurich_testv2_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#             break
        wandb.log({'Test_Epoch_Loss': epoch_loss / len(test_loader)})
        
        scores = foggy_zurich_testv2_running_metrics.get_scores()
        for k, v in scores[0].items():
            wandb.log({'Test_{}'.format(k): v})
        wandb.log({'Test_IoU': list(scores[1].values())})

#     break
        